In [69]:
import os
import pandas as pd
import numpy as np

# 定义文件夹路径
data_dir = "../DATA/Labeled_data/UsingValidationData/"
valid_models = {"KNN", "1L3DTCNN", "2L3DTCNN", "2LCNN", "2L3DCNN", "T2L3DCNN", "LSTM", "GRU", "FCLTC", "FCCfC", "NCPLTC", "NCPCfC"}
gesture_mapping = {"NC": 0, "ST": 1, "P": 2, "G": 3}

# 存储结果
data_results = []

# 遍历文件夹
def process_file(file_path, folder_name, model_name, gesture_label):
    df = pd.read_csv(file_path)
    length = len(df)
    gt_idx = gesture_mapping.get(gesture_label, -1)

    # 计算延迟时间
    Time_label, Time_model = None, None
    index_label_time, index_model_time = None, None

    for i in range(length):
        if df.loc[i, "label"] == 1:
            Time_label = df.loc[i, "time"]
            index_label_time = i
            break

    for i in range(length):
        if df.loc[i, "touch_type_idx"] != 0:
            Time_model = df.loc[i, "model_result_time"]
            index_model_time = i
            break

    if Time_label is None or Time_model is None:
        return

    T_delay = Time_model - Time_label
    index_diff = index_model_time - index_label_time

    # 计算准确率
    correct_predictions = []
    for i in range(length - index_diff):
        if i + index_diff >= length:
            break
        label_i = df.loc[i, "label"]
        touch_type_idx_iT = df.loc[i + index_diff, "touch_type_idx"]
        if (label_i == 0 and touch_type_idx_iT == 0) or (label_i == 1 and touch_type_idx_iT == gt_idx):
            correct_predictions.append(1)
        else:
            correct_predictions.append(0)
    
    accuracy = sum(correct_predictions) / len(correct_predictions) if correct_predictions else 0
    
    # 计算抖动次数
    jitter_count = np.sum(np.diff(df['touch_type_idx'].to_numpy()) != 0)-20
    
    data_results.append([folder_name, model_name, accuracy, T_delay, jitter_count])

# 遍历目录
for folder_name in os.listdir(data_dir):
    if "-" in folder_name:
        continue
    parts = folder_name.split("_")
    # print(parts)
    if parts[1] in valid_models and parts[-1] in gesture_mapping:
        model_name = parts[1]
        gesture_label = parts[-1]
        file_path = os.path.join(data_dir, folder_name, "labeled_modelResult_data.csv")
        if os.path.exists(file_path):
            process_file(file_path, folder_name, model_name, gesture_label)

out_file = os.path.join(data_dir, "processed_results.csv")
if os.path.exists(out_file):
    os.remove(out_file)
    print("delete")

# 保存结果到 CSV
results_df = pd.DataFrame(data_results, columns=["folder_name", "model_name", "accuracy", "delayed_time", "jitter_count"])
results_df = results_df.sort_values(by='model_name')
results_df.to_csv(out_file, index=False)

print("Processing complete. Results saved to processed_results.csv")


delete
Processing complete. Results saved to processed_results.csv


In [70]:
from IPython.core.display import display, HTML
result = pd.read_csv("../DATA/Labeled_data/UsingValidationData/processed_results.csv")
display(HTML(result.to_html(max_rows=100))) 
model_performance_summary = result.groupby('model_name')[["accuracy","delayed_time","jitter_count"]].mean()
display(HTML(model_performance_summary.to_html(max_rows=100))) 

out_file = os.path.join(data_dir, "model_performance_summary.csv")
if os.path.exists(out_file):
    os.remove(out_file)
    print("delete")

# 保存结果到 CSV
results_df = pd.DataFrame(model_performance_summary)
results_df = model_performance_summary.sort_values(by='model_name')
results_df.to_csv(out_file)
print("Processing complete. Results saved to model_performance_summary.csv")

/tmp/ipykernel_2154026/3149936885.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,folder_name,model_name,accuracy,delayed_time,jitter_count
0,0220_1L3DTCNN_Post123_T4ES_P,1L3DTCNN,0.882187,0.039400,25
1,0220_1L3DTCNN_Post123_T4ES_ST,1L3DTCNN,0.871944,0.071043,46
2,0220_1L3DTCNN_Post123_T4ES_G,1L3DTCNN,0.829739,0.150713,26
3,0220_2L3DCNN_Post123_T4ES_G,2L3DCNN,0.930943,0.022169,44
4,0220_2L3DCNN_Post123_T4ES_P,2L3DCNN,0.928076,0.064936,47
5,0220_2L3DCNN_Post123_T4ES_ST,2L3DCNN,0.926241,0.059068,42
6,0220_2LCNN_Post123_T4ES_ST,2LCNN,0.894187,0.066243,60
7,0220_2LCNN_Post123_T4ES_P,2LCNN,0.904994,0.066186,30
8,0220_2LCNN_Post123_T4ES_G,2LCNN,0.914080,0.024239,58
9,0220_GRU_Post123_T4ES_G,GRU,0.857240,0.045464,68


,accuracy,delayed_time,jitter_count
model_name,,,
1L3DTCNN,0.861290,0.087052,32.333333
2L3DCNN,0.928420,0.048724,44.333333
2LCNN,0.904420,0.052223,49.333333
GRU,0.883065,0.032450,46.000000
LSTM,0.872207,0.058491,63.333333
NCPCfC,0.859200,0.077710,65.000000
T2L3DCNN,0.912455,0.027864,30.666667


delete
Processing complete. Results saved to model_performance_summary.csv


In [76]:
display(HTML(model_performance_summary.sort_values(by='accuracy',ascending=False).to_html(max_rows=100))) 
display(HTML(model_performance_summary.sort_values(by='jitter_count',ascending=True).to_html(max_rows=100))) 
display(HTML(model_performance_summary.sort_values(by='delayed_time',ascending=True).to_html(max_rows=100))) 

,accuracy,delayed_time,jitter_count
model_name,,,
2L3DCNN,0.928420,0.048724,44.333333
T2L3DCNN,0.912455,0.027864,30.666667
2LCNN,0.904420,0.052223,49.333333
GRU,0.883065,0.032450,46.000000
LSTM,0.872207,0.058491,63.333333
1L3DTCNN,0.861290,0.087052,32.333333
NCPCfC,0.859200,0.077710,65.000000


,accuracy,delayed_time,jitter_count
model_name,,,
T2L3DCNN,0.912455,0.027864,30.666667
1L3DTCNN,0.861290,0.087052,32.333333
2L3DCNN,0.928420,0.048724,44.333333
GRU,0.883065,0.032450,46.000000
2LCNN,0.904420,0.052223,49.333333
LSTM,0.872207,0.058491,63.333333
NCPCfC,0.859200,0.077710,65.000000


,accuracy,delayed_time,jitter_count
model_name,,,
T2L3DCNN,0.912455,0.027864,30.666667
GRU,0.883065,0.032450,46.000000
2L3DCNN,0.928420,0.048724,44.333333
2LCNN,0.904420,0.052223,49.333333
LSTM,0.872207,0.058491,63.333333
NCPCfC,0.859200,0.077710,65.000000
1L3DTCNN,0.861290,0.087052,32.333333
